# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib.pyplot import imshow
from IPython.display import display

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms, models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transforms = train_transforms

test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform = train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform = valid_transforms)
test_data = datasets.ImageFolder(test_dir, transform = test_transforms)


# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloaders = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validloaders = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=True)
testloaders = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [28]:
import json
idx_to_class = {}

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    idx_to_class = cat_to_name
    
class_to_idx = {idx_to_class[idx]:idx for idx in idx_to_class}
# print (idx_to_class)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [5]:
from collections import OrderedDict

In [8]:
# TODO: Build and train your network
model = models.densenet121(pretrained=True)
# model
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
# classifier = nn.Sequential(OrderedDict([
#     ('fc1', nn.Linear(9216, 1000)),
#     ('relu', nn.ReLU()),
#     ('fc2', nn.Linear(1000, 102)),
#     ('output', nn.LogSoftmax(dim=1))
# ]))

classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 102)),
    ('output', nn.LogSoftmax(dim=1))
]))
                           
model.classifier = classifier
model.cuda()
model

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [9]:
#create loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [10]:
def validation (model, testloader, criterion):
    test_loss = 0
    accuracy = 0
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        loss = criterion(output, labels)
        
        test_loss += loss.item()
        
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    return test_loss, accuracy

In [11]:
epochs = 2
print_every = 20
steps = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in iter(trainloaders):
        steps += 1
        optimizer.zero_grad()

        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        
        loss = criterion(output, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps%print_every == 0:
            model.eval()
            with torch.no_grad():
                test_loss, accuracy = validation(model, testloaders, criterion)
            
            print("Epoch: {}/{} Training Loss: {:.4f} Test Loss: {:.3f} Test Accuracy: {:.3f}".format(epoch+1, epochs, running_loss/print_every, test_loss/len(testloaders), accuracy/len(testloaders)))
        
            running_loss = 0
            model.train()
        
        

Epoch: 1/2 Training Loss: 4.4359 Test Loss: 4.080 Test Accuracy: 0.131
Epoch: 1/2 Training Loss: 3.6994 Test Loss: 3.520 Test Accuracy: 0.274
Epoch: 1/2 Training Loss: 3.2032 Test Loss: 3.061 Test Accuracy: 0.407
Epoch: 1/2 Training Loss: 2.6653 Test Loss: 2.697 Test Accuracy: 0.511
Epoch: 1/2 Training Loss: 2.3114 Test Loss: 2.387 Test Accuracy: 0.606
Epoch: 2/2 Training Loss: 1.6831 Test Loss: 2.162 Test Accuracy: 0.601
Epoch: 2/2 Training Loss: 1.7616 Test Loss: 1.967 Test Accuracy: 0.674
Epoch: 2/2 Training Loss: 1.5761 Test Loss: 1.805 Test Accuracy: 0.693
Epoch: 2/2 Training Loss: 1.4675 Test Loss: 1.684 Test Accuracy: 0.720
Epoch: 2/2 Training Loss: 1.3388 Test Loss: 1.588 Test Accuracy: 0.721


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [13]:
# TODO: Do validation on the test set
test_loss, accuracy = validation(model, validloaders, criterion)
print ("Test Loss: {:.4f} Accuracy: {:.3f}".format(test_loss/len(validloaders), accuracy/len(validloaders)))

Test Loss: 1.2677 Accuracy: 0.789


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [17]:
# TODO: Save the checkpoint 
checkpoint = {
    'input_size': 1024,
    'output_size': 102,
    'optimizer' : optimizer.state_dict(),
    'class_to_idx': class_to_idx,
    'idx_to_class' : idx_to_class,
    'state_dict': model.state_dict()
}
# print(idx_to_class)

torch.save(checkpoint, 'image_classifier.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [18]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
idx_to_class = {}
class_to_idx = {}
def loadModel(chkpoint_name):
    checkpoint = torch.load(chkpoint_name)
    model = models.densenet121(pretrained=False)
    
    for param in model.parameters():
        param.requires_grad = False
    
    classifier = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(checkpoint['input_size'], checkpoint['output_size'])),
        ('output', nn.LogSoftmax(dim=1))
    ]))
    
    model.classifier = classifier 
    model.load_state_dict(checkpoint['state_dict'])
    model.optimizer = checkpoint['optimizer']
#     model.cuda()
    
    idx_to_class = checkpoint['idx_to_class']
    class_to_idx = checkpoint['class_to_idx']
    
    return model

model = loadModel('image_classifier.pth')
# idx_to_class

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [19]:
from sklearn.preprocessing import normalize

In [20]:
from numpy import asarray
def normalize1 (im):
    mean = [0.485, 0.456, 0.406]
    sd = [0.229, 0.224, 0.225]
    
    np_image = asarray(im)
    np_image = np_image.astype('float32')
    
    np_image /= 255.0
    
    np_image = (np_image - mean) / sd
    return np_image

# process_image('./flowers/test/43/image_02365.jpg')

In [21]:
def resize (image):
    aspect_ratio = image.size[0]/image.size[1]
    
    return image.resize((int(256*aspect_ratio), 256))

new_width, new_height = 224, 224
def crop (image):
    width, height = image.size
    
    left = (width - new_width)/2
    top = (height - new_height)/2    
    right = (width + new_width)/2 
    bottom = (height + new_height)/2
    
    return image.crop((left, top, right, bottom))

In [22]:
from PIL import Image
from numpy import asarray

def process_image(image_path):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    
    im = Image.open(image_path)
#     display(im)
#     im = resize(im)
#     im = crop(im)
#     display(im)
    
#     np_array = normalize1(im)
#     im = torch.from_numpy(np_array.transpose())

    
    
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    processed_image = preprocess(im)
#     print(processed_image.size)
#     imshow(processed_image)
    return processed_image
#     return asarray(processed_image)

#     return np_array


process_image('./flowers/test/43/image_02365.jpg')


tensor([[[ 1.9749,  1.9749,  2.0777,  ..., -1.3302, -1.2103, -1.1760],
         [ 1.9578,  1.9920,  2.1290,  ..., -1.2617, -1.1760, -1.1418],
         [ 1.9749,  1.9749,  2.0434,  ..., -1.2103, -1.1760, -1.1247],
         ...,
         [-1.3473, -1.3815, -1.3815,  ...,  0.6906,  0.4508,  0.1939],
         [-1.3473, -1.3815, -1.3815,  ...,  0.3138,  0.1254,  0.0227],
         [-1.3644, -1.3815, -1.3815,  ...,  0.0912,  0.1426,  0.2796]],

        [[ 0.2577,  0.0476,  0.0651,  ..., -0.3901, -0.4601, -0.4251],
         [ 0.0826, -0.0399,  0.0826,  ..., -0.4251, -0.4251, -0.3901],
         [-0.0924, -0.1975, -0.0049,  ..., -0.4601, -0.4251, -0.3725],
         ...,
         [-0.5651, -0.5651, -0.6001,  ..., -0.0224, -0.2325, -0.5126],
         [-0.5651, -0.5651, -0.6001,  ..., -0.3375, -0.5301, -0.6352],
         [-0.5826, -0.5826, -0.6001,  ..., -0.5126, -0.4951, -0.3901]],

        [[ 1.3328,  1.2282,  1.2980,  ..., -1.6999, -1.5953, -1.5604],
         [ 1.2282,  1.1934,  1.3328,  ..., -1

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [23]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [32]:

def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    image = process_image(image_path)
    
    with torch.no_grad():
        image.unsqueeze_(0)
        output = model.forward(image)
    
    ps = torch.exp(output)
    probs, indices = ps.topk(topk)
    
    probs = probs.numpy()[0]
    indices  = indices.numpy()[0]
    classes = [idx_to_class[str(x)] for x in indices]
    return probs, classes

# predict(full_path, model, 5)
# ps, labels = predict(full_path, model, 5)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [25]:
def assess_pred (ps, labels, image_path):
    name_arr = image_path.split("/")
    
    index_max = np.argmax(ps)
    
    
    actual = idx_to_class.get(str(name_arr[-2]))
    predicted = labels[index_max]
    accuracy = max(ps)
#     print(labels, index_max, actual, predicted, accuracy)
    
    return actual, predicted, accuracy
    

In [26]:
def graph (x, y, image, x_label="x", y_label="y", topk=5 ):
    plt.rcdefaults()
    fig, ax = plt.subplots()
    
    y_pos = np.arange(topk)
    performance = x
    
    plt.bar(y_pos, performance, align='center', alpha=0.5)
    labels = [idx_to_class.get(str(idx)) for idx in iter(y)]
    
    plt.xticks(y_pos, labels)
    display(resize(Image.open(image)))
    
    
#     plt.imshow(Image.open(image).numpy().squeeze(), cmap='Greys_r');
#     ax.imshow(image)
    plt.show()
 


In [30]:
# TODO: Display an image along with the top 5 classes
# import glob



import os
root_path = "./flowers/test"
test_folder= os.listdir (root_path)
# print(test_folder)

count = 0
break_at = 10
for class_path in test_folder:
    count += 1
#     print(class_path)
    image_path = os.listdir ("{}/{}".format(root_path, class_path))[0]
    full_path = "{}/{}/{}".format(root_path, class_path, image_path)
    ps, labels = predict(full_path, model, 5)
    actual, predicted, accuracy = assess_pred(ps, labels, full_path)
    print("Actual: {} ... Predicted: {} ... Probability: {:.3f} ... Filename: {}".format(actual, predicted, accuracy, image_path))
    
#     graph(ps, labels, full_path)
    if (count == break_at):
        break
        
        

# print(ps, labels)
# graph(ps, labels, images[0])

Actual: sword lily ... Predicted: columbine ... Probability: 0.032 ... Filename: image_02365.jpg
Actual: bee balm ... Predicted: columbine ... Probability: 0.031 ... Filename: image_03054.jpg
Actual: cyclamen ... Predicted: columbine ... Probability: 0.031 ... Filename: image_00477.jpg
Actual: hippeastrum ... Predicted: columbine ... Probability: 0.031 ... Filename: image_04860.jpg
Actual: bougainvillea ... Predicted: columbine ... Probability: 0.032 ... Filename: image_07573.jpg
Actual: artichoke ... Predicted: columbine ... Probability: 0.031 ... Filename: image_04112.jpg
Actual: balloon flower ... Predicted: columbine ... Probability: 0.030 ... Filename: image_06197.jpg
Actual: desert-rose ... Predicted: columbine ... Probability: 0.033 ... Filename: image_04801.jpg
Actual: monkshood ... Predicted: columbine ... Probability: 0.031 ... Filename: image_06413.jpg
Actual: rose ... Predicted: columbine ... Probability: 0.031 ... Filename: image_01165.jpg
